In [3]:
import math
# Importing standard Qiskit libraries and configuring account
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
IBMQ.load_account() 
backend = BasicAer.get_backend('unitary_simulator')
provider = IBMQ.get_provider(hub='ibm-q-research')



#%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit,  execute, BasicAer, Aer

#error mitigation toolbox
#from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

ibmqfactory.load_account:WARNING:2022-08-29 02:32:31,403: Credentials are already in use. The existing account in the session will be replaced.


In [6]:
#name I decided to give our 'quantum computer' in simulations
cname='ibmq_edendork'


In [7]:
import pandas as pd
filename = '4outcome_POVM.csv'
filename2 = '4outcome_est.csv'
data = pd.read_csv(filename, sep=",", header=None)
data = data.applymap(lambda s: np.complex(s.replace('i', 'j'))).values

data2 = pd.read_csv(filename2, sep=",", header=None)
#data2 = data2.applymap(lambda s: ).values


print(data[[0]])

datadag=data.conj().T

data2=np.array(data2)


/tmp/ipykernel_77/1267672327.py:5: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.applymap(lambda s: np.complex(s.replace('i', 'j'))).values


[[-3.89824411e-01+1.78833700e-03j  3.97786032e-01+1.54724660e-02j
   4.21416004e-04+1.68334768e-04j  6.14906101e-01+5.13039817e-03j
  -2.13202500e-04+2.88646828e-05j -6.58074047e-05+2.95180540e-04j
  -1.12063622e-04-3.90059843e-05j  1.29468152e-04+1.94755579e-05j
   7.12406470e-05-1.68835213e-04j  2.21234948e-05+1.58524548e-04j
   4.09588597e-04-3.29233540e-04j -5.17396866e-06-2.97844432e-04j
  -3.16162064e-03-2.16035052e-02j  5.09021262e-04+2.14657173e-02j
  -3.89328823e-01-1.22582240e-02j  3.98438895e-01-2.34071517e-03j]]


In [9]:

from scipy import stats
from scipy.optimize import fsolve

In [12]:
#data analysis
#two noise strings, one for large noise one for small
noisestring=str('aug22_noiseth0n0.001')
noisestring=str('aug22_noiseth0n0.005')
th=0

#first load training data to get the slopes and intercepts again

F = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_4Q_est_X")+str("/")+str("training_data")+str(".npz")))
FY = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_4Q_est_Y")+str("/")+str("training_data")+str(".npz")))


a=F['arr_0']
aY=FY['arr_0']
noisyx=np.transpose(a)[0]
truex=np.transpose(a)[1]
noisyy=np.transpose(aY)[0]
truey=np.transpose(aY)[1]


res4QX = stats.linregress(np.array(noisyx),np.array(truex))
res4QY = stats.linregress(np.array(noisyy),np.array(truey))

#parameters from above
cx=res4QX.intercept
mx=res4QX.slope
cy=res4QY.intercept
my=res4QY.slope



#then load data data to get the total number of counts in each for bootstrapping to get the error bars
FN = np.load(str(cname+str("_LR")+str("/")+noisestring+str("_4Q_nvec")+str("/")+str("nvec_data")+str(".npz")))
a=FN['arr_0']



n0000T=0
n0001T=0
n0010T=0
n0011T=0
n0100T=0
n0101T=0
n0110T=0
n0111T=0
n1000T=0
n1001T=0
n1010T=0
n1011T=0
n1100T=0
n1101T=0
n1110T=0
n1111T=0

#get the real MSE here
msereal=[]
shotsin=256
for ii in range(1000):
    n0000=a[ii][0]
    n0000T=n0000T+a[ii][0]
    n0001=a[ii][1]
    n0001T=n0001T+a[ii][1]
    n0010=a[ii][2]
    n0010T=n0010T+a[ii][2]
    n0011=a[ii][3]
    n0011T=n0011T+a[ii][3]
    n0100=a[ii][4]
    n0100T=n0100T+a[ii][4]
    n0101=a[ii][5]
    n0101T=n0101T+a[ii][5]
    n0110=a[ii][6]
    n0110T=n0110T+a[ii][6]
    n0111=a[ii][7]
    n0111T=n0111T+a[ii][7]
    n1000=a[ii][8]
    n1000T=n1000T+a[ii][8]
    n1001=a[ii][9]
    n1001T=n1001T+a[ii][9]
    n1010=a[ii][10]
    n1010T=n1010T+a[ii][10]
    n1011=a[ii][11]
    n1011T=n1011T+a[ii][11]
    n1100=a[ii][12]
    n1100T=n1100T+a[ii][12]
    n1101=a[ii][13]
    n1101T=n1101T+a[ii][13]
    n1110=a[ii][14]
    n1110T=n1110T+a[ii][14]
    n1111=a[ii][15]
    n1111T=n1111T+a[ii][15]
    
    estx=(n0000*data2[0][0]+n0001*data2[0][1]+n0010*data2[0][2]+n0011*data2[0][3]+n0100*data2[0][4]+n0101*data2[0][5]+n0110*data2[0][6]+n0111*data2[0][7]+n1000*data2[0][8]+n1001*data2[0][9]+n1010*data2[0][10]+n1011*data2[0][11]+n1100*data2[0][12]+n1101*data2[0][13]+n1110*data2[0][14]+n1111*data2[0][15])/shotsin
    esty=(n0000*data2[1][0]+n0001*data2[1][1]+n0010*data2[1][2]+n0011*data2[1][3]+n0100*data2[1][4]+n0101*data2[1][5]+n0110*data2[1][6]+n0111*data2[1][7]+n1000*data2[1][8]+n1001*data2[1][9]+n1010*data2[1][10]+n1011*data2[1][11]+n1100*data2[1][12]+n1101*data2[1][13]+n1110*data2[1][14]+n1111*data2[1][15])/shotsin
    estyA=esty*my+cy
    estxA=estx*mx+cx
    msereal.append(estxA**2+estyA**2)

ntot=n0000T+n0001T+n0010T+n0011T+n0100T+n0101T+n0110T+n0111T+n1000T+n1001T+n1010T+n1011T+n1100T+n1101T+n1110T+n1111T

p0000=n0000T/ntot
p0001=n0001T/ntot
p0010=n0010T/ntot
p0011=n0011T/ntot
p0100=n0100T/ntot
p0101=n0101T/ntot
p0110=n0110T/ntot
p0111=n0111T/ntot

p1000=n1000T/ntot
p1001=n1001T/ntot
p1010=n1010T/ntot
p1011=n1011T/ntot
p1100=n1100T/ntot
p1101=n1101T/ntot
p1110=n1110T/ntot
p1111=n1111T/ntot

mseT=[]

for hh in range(500):
    mseint=[]
    if ii%100==0:
        print('Bootstrap run #',ii)
    for kk in range(1000):

        rv= np.random.multinomial(shotsin, [p0000,p0001,p0010,p0011,p0100,p0101,p0110,p0111,p1000,p1001,p1010,p1011,p1100,p1101,p1110,p1111], size=1)
        n0000=rv[0][0]
        n0001=rv[0][1]
        n0010=rv[0][2]
        n0011=rv[0][3]
        n0100=rv[0][4]
        n0101=rv[0][5]
        n0110=rv[0][6]
        n0111=rv[0][7]
        
        n1000=rv[0][8]
        n1001=rv[0][9]
        n1010=rv[0][10]
        n1011=rv[0][11]
        n1100=rv[0][12]
        n1101=rv[0][13]
        n1110=rv[0][14]
        n1111=rv[0][15]
        
        estx=(n0000*data2[0][0]+n0001*data2[0][1]+n0010*data2[0][2]+n0011*data2[0][3]+n0100*data2[0][4]+n0101*data2[0][5]+n0110*data2[0][6]+n0111*data2[0][7]+n1000*data2[0][8]+n1001*data2[0][9]+n1010*data2[0][10]+n1011*data2[0][11]+n1100*data2[0][12]+n1101*data2[0][13]+n1110*data2[0][14]+n1111*data2[0][15])/shotsin
        esty=(n0000*data2[1][0]+n0001*data2[1][1]+n0010*data2[1][2]+n0011*data2[1][3]+n0100*data2[1][4]+n0101*data2[1][5]+n0110*data2[1][6]+n0111*data2[1][7]+n1000*data2[1][8]+n1001*data2[1][9]+n1010*data2[1][10]+n1011*data2[1][11]+n1100*data2[1][12]+n1101*data2[1][13]+n1110*data2[1][14]+n1111*data2[1][15])/shotsin
        estyA=esty*my+cy
        estxA=estx*mx+cx
        mseint.append(estxA**2+estyA**2)
        
    mseT.append(np.mean(mseint))
    
print('Real MSE = ',np.mean(msereal))
print('Bootstrapped MSE = ',np.mean(mseT),'+/-',np.std(mseT))

RealMSE =  0.10503203031307323
Bootstrapped MSE =  0.10135740690754727 +/- 0.0031665213123904306
